## Preprocessing - make features/columns 

### Imports and read in data

In [1]:
### imports
import pandas as pd # DataFrame Manipulation Package
import numpy as np
import matplotlib.pyplot as plt
import string

from sklearn.feature_extraction.text import TfidfVectorizer # Convert a collection of raw documents to a matrix of TF-IDF features
from sklearn.decomposition import LatentDirichletAllocation # Latent Dirichlet Allocation is a topic model that is used for discovering abstract topics from a collection of documents (variational Bayes algorithm)
from sklearn.feature_extraction.text import CountVectorizer # Convert a collection of text documents to a matrix of token counts

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB # The multinomial Naive Bayes classifier is suitable for classification with discrete features (e.g., word counts for text classification)

import string # Collection of string operations
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer #Lemmatize using WordNet's built-in morphy function. Returns the input word unchanged if it cannot be found in WordNet.
from nltk import word_tokenize

from nltk.sentiment.util import mark_negation

In [2]:
### read in unlabed data

unlabeled_df =  pd.read_csv('/home/jack/code/jackoutthebox/adverse_drug_reactions/raw_data/drugsComTrain_raw.csv', nrows=100)
unlabeled_df.tail()

,uniqueID,drugName,condition,review,rating,date,usefulCount
95,45237,Fluoxetine,Major Depressive Disorde,"""I started Prozac as one of my first anti depr...",2,12-Jan-16,18
96,102810,Aripiprazole,Depression,"""Intake Effexor XR 375 mg, and lorazepam for d...",4,17-Aug-12,33
97,60280,NuvaRing,Birth Control,"""I am torn by the Nuvaring. The convenience is...",5,31-Oct-11,0
98,10677,Spironolactone,Acne,"""I&#039;m 30 years old. I started having real...",9,21-Aug-13,31
99,196244,Fluvoxamine,Anxiety and Stress,"""I&#039;ve suffered from panic attacks and anx...",9,3-Jan-11,44


In [3]:
### read in labeled data from Hendrike

labeled_df =  pd.read_csv('/home/jack/code/jackoutthebox/adverse_drug_reactions/raw_data/adr_labelled_data.csv')
labeled_df
labeled_df.head()


,uniqueID,drugName,condition,review,rating,date,usefulCount,sideEffect
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27,0.0
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192,1.0
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17,1.0
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,3-Nov-15,10,1.0
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37,1.0


### import list of side effects

In [4]:
### import list of side effects

from csv import reader
# read csv file as a list of lists
with open('../raw_data/frequent_adr.csv', 'r') as read_obj:
    # pass the file object to reader() to get the reader object
    csv_reader = reader(read_obj)
    # Pass reader object to list() to get a list of lists
    side_effects = list(csv_reader)

In [5]:
side_effects.pop(0)

['0']

In [6]:
print(type(side_effects))

<class 'list'>


In [7]:
side_effect_df = pd.DataFrame(side_effects)
side_effect_df.head()

,0
0,Abdominal pain
1,Gastrointestinal pain
2,Amblyopia
3,Anaemia
4,Anorexia


In [8]:
si_list = [] 
for sublist in side_effects: 
    for item in sublist: si_list.append(item)

print(type(si_list))
#si_list

<class 'list'>


In [9]:
df_sideeffects = side_effect_df[0].str.split(expand=True).stack().value_counts()
df_sideeffects.head()

disorder     92
increased    66
site         54
pain         51
infection    40
dtype: int64

### Preprocessing


In [10]:
## functions

#stop_words
stop_words = set(stopwords.words('english')) 

for negation in ["no", "not", "shouldn't", "aren't", "couldn't", "didn't", "doesn't", "don't", "wasn't", "weren't", "wouldn't"]:
    stop_words.remove(negation)

#stop_words_with side effects

stop_words_se = set(stopwords.words('english')) 

for negation in ["no", "not", "shouldn't", "aren't", "couldn't", "didn't", "doesn't", "don't", "wasn't", "weren't", "wouldn't"]:
    stop_words_se.remove(negation)

for effect in si_list: 
    stop_words_se.add(effect) 
    
    

def to_list(x):
    list_words = x.split(' ')
    return list_words

def to_string(x):
    string = " ".join(x)
    return string

#===============================================================

def punctuation(x):
    for punctuation in string.punctuation:
        x =  x.replace(punctuation, '')
    return x.lower()

def remove_numbers (x):
    words_only = ''.join([i for i in x if not i.isdigit()])
    return words_only

def m_negation(x):
    tokenized = word_tokenize(x)
    without_stopwords = [word for word in tokenized if not word in stop_words]
    tokenized_neg = mark_negation(without_stopwords)
    return tokenized_neg

def remove_stopwords(x):
    tokenized = word_tokenize(x)
    without_stopwords = [word for word in tokenized if not word in stop_words]
    return without_stopwords

def remove_stopwords_se(x):
    tokenized = word_tokenize(x)
    without_stopwords = [word for word in tokenized if not word in stop_words_se]
    return without_stopwords

def m_negation_se(x):
    tokenized = word_tokenize(x)
    without_stopwords = [word for word in tokenized if not word in stop_words_se]
    tokenized_neg = mark_negation(without_stopwords)
    return tokenized_neg

def lemmatize_review(x):
    lemma = WordNetLemmatizer()
    lista = []
    for w in x:
       lista.append(lemma.lemmatize(w))
    return lista

#===============================================================

def count_words(x):
    wordfreq = []
    for w in x:
        wordfreq.append(x.count(w))
    return dict(zip(x, wordfreq))

def total_count(x):
    total_count = {}
    for row in x:
        for key in row.keys():
          if key in total_count:
              total_count[key] += 1
          else:
              total_count[key] = 1
    return pd.DataFrame(sorted(total_count.items(), key=lambda x: x[1], reverse=True)).head(30).T

#===============================================================

def print_topics(model, vectorizer):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], round(topic[i], 2))
                        for i in topic.argsort()[:-10 - 1:-1]])

In [11]:
### enter in juan prepreprocessing - stemming, lemmitization, etc.

unlabeled_df["clean_review"] = unlabeled_df["review"].apply(punctuation)
unlabeled_df['clean_review'] = unlabeled_df.clean_review.apply(remove_numbers)
unlabeled_df['clean_review_lst'] = unlabeled_df.clean_review.apply(to_list)

unlabeled_df["nonStopwords_review_lst"] = unlabeled_df.clean_review.apply(remove_stopwords)
unlabeled_df["nonStopwords_review_str"] = unlabeled_df.nonStopwords_review_lst.apply(to_string)

unlabeled_df["nonStopwords_review_lst_MN"] = unlabeled_df.clean_review.apply(m_negation)
unlabeled_df["nonStopwords_review_str_MN"] = unlabeled_df.nonStopwords_review_lst_MN.apply(to_string)

unlabeled_df["lemmatized_review_lst"] = unlabeled_df.nonStopwords_review_lst_MN.apply(lemmatize_review)
unlabeled_df["lemmatized_review_str"] = unlabeled_df.lemmatized_review_lst.apply(to_string)

unlabeled_df["lemmatized_review"] = unlabeled_df.nonStopwords_review_lst.apply(lemmatize_review)
unlabeled_df["lemmatized_review"] = unlabeled_df.lemmatized_review.apply(to_string)

unlabeled_df["words_count"] = unlabeled_df.lemmatized_review_lst.apply(count_words)

unlabeled_df.head(3)



,uniqueID,drugName,condition,review,rating,date,usefulCount,clean_review,clean_review_lst,nonStopwords_review_lst,nonStopwords_review_str,nonStopwords_review_lst_MN,nonStopwords_review_str_MN,lemmatized_review_lst,lemmatized_review_str,lemmatized_review,words_count
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27,it has no side effect i take it in combination...,"[it, has, no, side, effect, i, take, it, in, c...","[no, side, effect, take, combination, bystolic...",no side effect take combination bystolic mg fi...,"[no, side_NEG, effect_NEG, take_NEG, combinati...",no side_NEG effect_NEG take_NEG combination_NE...,"[no, side_NEG, effect_NEG, take_NEG, combinati...",no side_NEG effect_NEG take_NEG combination_NE...,no side effect take combination bystolic mg fi...,"{'no': 1, 'side_NEG': 1, 'effect_NEG': 1, 'tak..."
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192,my son is halfway through his fourth week of i...,"[my, son, is, halfway, through, his, fourth, w...","[son, halfway, fourth, week, intuniv, became, ...",son halfway fourth week intuniv became concern...,"[son, halfway, fourth, week, intuniv, became, ...",son halfway fourth week intuniv became concern...,"[son, halfway, fourth, week, intuniv, became, ...",son halfway fourth week intuniv became concern...,son halfway fourth week intuniv became concern...,"{'son': 1, 'halfway': 1, 'fourth': 1, 'week': ..."
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17,i used to take another oral contraceptive whic...,"[i, used, to, take, another, oral, contracepti...","[used, take, another, oral, contraceptive, pil...",used take another oral contraceptive pill cycl...,"[used, take, another, oral, contraceptive, pil...",used take another oral contraceptive pill cycl...,"[used, take, another, oral, contraceptive, pil...",used take another oral contraceptive pill cycl...,used take another oral contraceptive pill cycl...,"{'used': 1, 'take': 1, 'another': 1, 'oral': 1..."


## create vectorized dataframe without side effects (include side effects as stopwords)

In [12]:
### read in unlabed data

unlabeled_no_sideseffects_df =  pd.read_csv('/home/jack/code/jackoutthebox/adverse_drug_reactions/raw_data/drugsComTrain_raw.csv', nrows=100)
unlabeled_no_sideseffects_df.tail()

,uniqueID,drugName,condition,review,rating,date,usefulCount
95,45237,Fluoxetine,Major Depressive Disorde,"""I started Prozac as one of my first anti depr...",2,12-Jan-16,18
96,102810,Aripiprazole,Depression,"""Intake Effexor XR 375 mg, and lorazepam for d...",4,17-Aug-12,33
97,60280,NuvaRing,Birth Control,"""I am torn by the Nuvaring. The convenience is...",5,31-Oct-11,0
98,10677,Spironolactone,Acne,"""I&#039;m 30 years old. I started having real...",9,21-Aug-13,31
99,196244,Fluvoxamine,Anxiety and Stress,"""I&#039;ve suffered from panic attacks and anx...",9,3-Jan-11,44


In [13]:
unlabeled_no_sideseffects_df["clean_review"] = unlabeled_no_sideseffects_df["review"].apply(punctuation)
unlabeled_no_sideseffects_df['clean_review'] = unlabeled_no_sideseffects_df.clean_review.apply(remove_numbers)
unlabeled_no_sideseffects_df['clean_review_lst'] = unlabeled_no_sideseffects_df.clean_review.apply(to_list)

unlabeled_no_sideseffects_df["nonStopwords_review_lst"] = unlabeled_no_sideseffects_df.clean_review.apply(remove_stopwords_se)
unlabeled_no_sideseffects_df["nonStopwords_review_str"] = unlabeled_no_sideseffects_df.nonStopwords_review_lst.apply(to_string)

unlabeled_no_sideseffects_df["nonStopwords_review_lst_MN"] = unlabeled_no_sideseffects_df.clean_review.apply(m_negation_se)
unlabeled_no_sideseffects_df["nonStopwords_review_str_MN"] = unlabeled_no_sideseffects_df.nonStopwords_review_lst_MN.apply(to_string)

unlabeled_no_sideseffects_df["lemmatized_review_lst"] = unlabeled_no_sideseffects_df.nonStopwords_review_lst_MN.apply(lemmatize_review)
unlabeled_no_sideseffects_df["lemmatized_review_str"] = unlabeled_no_sideseffects_df.lemmatized_review_lst.apply(to_string)

unlabeled_no_sideseffects_df["lemmatized_review"] = unlabeled_no_sideseffects_df.nonStopwords_review_lst.apply(lemmatize_review)
unlabeled_no_sideseffects_df["lemmatized_review"] = unlabeled_no_sideseffects_df.lemmatized_review.apply(to_string)

unlabeled_no_sideseffects_df["words_count"] = unlabeled_no_sideseffects_df.lemmatized_review_lst.apply(count_words)

unlabeled_no_sideseffects_df.head(3)

,uniqueID,drugName,condition,review,rating,date,usefulCount,clean_review,clean_review_lst,nonStopwords_review_lst,nonStopwords_review_str,nonStopwords_review_lst_MN,nonStopwords_review_str_MN,lemmatized_review_lst,lemmatized_review_str,lemmatized_review,words_count
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27,it has no side effect i take it in combination...,"[it, has, no, side, effect, i, take, it, in, c...","[no, side, effect, take, combination, bystolic...",no side effect take combination bystolic mg fi...,"[no, side_NEG, effect_NEG, take_NEG, combinati...",no side_NEG effect_NEG take_NEG combination_NE...,"[no, side_NEG, effect_NEG, take_NEG, combinati...",no side_NEG effect_NEG take_NEG combination_NE...,no side effect take combination bystolic mg fi...,"{'no': 1, 'side_NEG': 1, 'effect_NEG': 1, 'tak..."
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192,my son is halfway through his fourth week of i...,"[my, son, is, halfway, through, his, fourth, w...","[son, halfway, fourth, week, intuniv, became, ...",son halfway fourth week intuniv became concern...,"[son, halfway, fourth, week, intuniv, became, ...",son halfway fourth week intuniv became concern...,"[son, halfway, fourth, week, intuniv, became, ...",son halfway fourth week intuniv became concern...,son halfway fourth week intuniv became concern...,"{'son': 1, 'halfway': 1, 'fourth': 1, 'week': ..."
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17,i used to take another oral contraceptive whic...,"[i, used, to, take, another, oral, contracepti...","[used, take, another, oral, contraceptive, pil...",used take another oral contraceptive pill cycl...,"[used, take, another, oral, contraceptive, pil...",used take another oral contraceptive pill cycl...,"[used, take, another, oral, contraceptive, pil...",used take another oral contraceptive pill cycl...,used take another oral contraceptive pill cycl...,"{'used': 1, 'take': 1, 'another': 1, 'oral': 1..."


## create vectorized side effects dataframe - orginal df minus df without side effects

### Make columns for whether side effects are mentioned in a review

In [14]:
### take sides effects from dictionary and create column in dataframe

Dummy_side_effects= {
    'abdominal': ['constipation', 'diarrhea'],
    'skin' : ['rash'],
    'vertigo' : ['dizziness', 'drowsiness'],
    'headache' : ['headache'],
    'mood disorders' : ['insomnia', 'mood swings']}

def make_columns(SideEffectdict):
    for condition, words in SideEffectdict.items():
        #f"unlabed_df['{key}'] = unlabed_df['clean_review'].str.contains('{value}')"
        for word in words:
            contains_key = unlabed_df['clean_review'].str.contains()
            df[key] = 
    return unlabed_df
    

make_columns(Dummy_side_effects)



SyntaxError: invalid syntax (<ipython-input-14-34ffcc5c06c3>, line 15)

# vectorize words based on lemmitzied reveiws 

### vectorize unlabeled df with bag of words

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

X = vectorizer.fit_transform(unlabeled_df["lemmatized_review"])

X.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [16]:
X_df  = pd.DataFrame(X.toarray())

#X_df[0] = X_df[0].apply(vectorizer.get_feature_names())
X_df.head()

,0,1,2,3,4,5,6,7,8,9,...,1544,1545,1546,1547,1548,1549,1550,1551,1552,1553
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0


In [17]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

X_noSE = vectorizer.fit_transform(unlabeled_no_sideseffects_df["review"])

X_noSE.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [2, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 2, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [18]:
X.shape, X_noSE.shape

((100, 1554), (100, 1779))

In [19]:
list_merge_noSE = ['juan', 'jack', 'peter', 'hendrike']
list_merge_SE = ['juan', 'jack', 'peter', 'hendrike', 'headache']

def create_strings(x):
    lista = []
    y = remove_stopwords_se(x)
    for i in x:
        if i not in y:
            lista.append(i)
    return lista

create_strings(list_merge_SE)
        


TypeError: expected string or bytes-like object

In [20]:

X_SE = list(set(X_noSE()) - set(X))

TypeError: 'csr_matrix' object is not callable

In [21]:
X_df = pd.DataFrame(X)
X_noSE_df = pd.DataFrame(X_noSE)

X_df.head()
SE_df = X_noSE_df - X_df

ValueError: inconsistent shapes

### trying to vectorize with TDiF with a custom vocababuly. Not working...

In [22]:


Dummy_side_effects= {
    'constipation': 'abdominal',
    'diarrhea' : 'abdominal',
    'rash' : 'skin',
    'dizziness' : 'vertigo',
    'drowsiness' : 'vertigo',
    'headache' : 'headache',
    'insomnia' : 'mood disorders',
    'mood swings' : 'mood disorders'}

vectorizer = TfidfVectorizer(min_df = 0.01, 
                             max_df = 0.50, 
                             max_features = None,
                             vocabulary = None,
                             ngram_range = (2, 2)).fit(unlabeled_df["review"]) #

# MINDF Ignore terms that have a document frequency strictly higher than the given threshold
# MAXDF When building the vocabulary ignore terms that have a document frequency strictly lower than the given threshold

data_vectorized = vectorizer.transform(unlabeled_df["lemmatized_review"])


In [23]:
data_vectorized

<100x6534 sparse matrix of type '<class 'numpy.float64'>'
	with 1474 stored elements in Compressed Sparse Row format>

### vectorize words based on lemmitzied reveiws - unlabeled without side effects data set

In [24]:

Dummy_side_effects= {
    'constipation': 'abdominal',
    'diarrhea' : 'abdominal',
    'rash' : 'skin',
    'dizziness' : 'vertigo',
    'drowsiness' : 'vertigo',
    'headache' : 'headache',
    'insomnia' : 'mood disorders',
    'mood swings' : 'mood disorders'}

vectorizer = TfidfVectorizer(min_df = 0.01, 
                             max_df = 0.50, 
                             max_features = None,
                             vocabulary = None,
                             ngram_range = (2, 2)).fit(unlabeled_no_sideseffects_df["review"]) #

# MINDF Ignore terms that have a document frequency strictly higher than the given threshold
# MAXDF When building the vocabulary ignore terms that have a document frequency strictly lower than the given threshold

data_vectorized_se = vectorizer.transform(unlabeled_no_sideseffects_df["lemmatized_review"])

In [25]:
data_vectorized_se

<100x6534 sparse matrix of type '<class 'numpy.float64'>'
	with 1474 stored elements in Compressed Sparse Row format>

In [26]:
side_effects_vectorized = data_vectorized - data_vectorized_se

In [27]:
print(side_effects_vectorized)